In [ ]:
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import asyncio
import numpy as np
from sqlalchemy.orm import joinedload

from benchmarklib import BenchmarkDatabase
from rbf import RandomBooleanFunctionTrial, RandomBooleanFunction
from benchmarklib.compilers import CompileType, XAGCompiler

from qiskit_ibm_runtime import QiskitRuntimeService, RuntimeJobNotFound

import logging
from typing import Iterable, List, Tuple, Dict, Any, Union, Optional
import qiskit
from qiskit.providers import Backend
from qiskit import QuantumCircuit, transpile
import random

from sqlalchemy import select, func
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

from benchmarklib import BenchmarkDatabase
from benchmarklib import BatchQueue
from benchmarklib.compilers import SynthesisCompiler
from benchmarklib.core.database import BackendPropertyManager

In [ ]:
load_dotenv()
API_TOKEN_OLD = os.getenv("API_TOKEN_OLD")
API_INSTANCE_OLD = os.getenv("API_INSTANCE_OLD")
service = QiskitRuntimeService()  # default service with new credentials
service_old = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    token=API_TOKEN_OLD,
    instance=API_INSTANCE_OLD
)
backend = service.backend("ibm_rensselaer")
benchmark_db = BenchmarkDatabase("rbf.db", RandomBooleanFunction, RandomBooleanFunctionTrial)

### Backend Properties

In [ ]:
backend_db = BackendPropertyManager("rbf.db")

In [ ]:
properties = backend.properties()
error_rates = {}
for g in backend.configuration().basis_gates:
    error_rates[g] = []
    for gate_info in properties.gates:
        d = gate_info.to_dict()
        if d["gate"] == g:
            for param in d["parameters"]:
                if param["name"] == "gate_error":
                    error_rates[g].append(param["value"])
print("Current backend gate error rates:")             
for g in error_rates:
    if len(error_rates[g]) > 0:
        print(f"{g}: avg error = {np.mean(error_rates[g]):.2e} (min={np.min(error_rates[g]):.2e}, max={np.max(error_rates[g]):.2e})")

In [ ]:
import datetime

cache = {}

def get_mean_gate_errors(at: datetime.date):
    if at in cache:
        return cache[at]
    props = backend_db.latest(backend, at)
    cache[at] = props.get_average_gate_errors()
    return cache[at]

med_cache = {}
def get_median_gate_errors(at: datetime.date):
    if at in med_cache:
        return med_cache[at]
    props = backend_db.latest(backend, at)
    med_cache[at] = {g : np.median(v) for g, v in props.get_gate_errors().items()}
    return med_cache[at]

In [ ]:
with benchmark_db.session() as session:
    query = (select(RandomBooleanFunction.num_vars, RandomBooleanFunction.complexity)
            .select_from(RandomBooleanFunctionTrial).join(RandomBooleanFunctionTrial.problem)
            .where(RandomBooleanFunctionTrial._circuit_qpy != None).limit(20000)
            )
    trials = session.execute(query).all()

In [ ]:
counts = {}
for num_vars, complexity in trials:
    counts[(num_vars, complexity)] = counts.get((num_vars, complexity), 0) + 1

# create matrix
max_vars = max(k[0] for k in counts.keys())
max_complexity = max(k[1] for k in counts.keys())
matrix = np.zeros((max_vars, max_complexity), dtype=int)
for (num_vars, complexity), count in counts.items():
    matrix[num_vars - 1, complexity - 1] = count
plt.imshow(matrix, cmap='Blues', interpolation='nearest')

### Analytic Success Rate Estimate

In [ ]:
def compute_accumulated_gate_error1(circuit, gate_errors):
    assert circuit is not None
    expected_success_rate = 1.0
    for inst in circuit.data:
        expected_success_rate *= (1 - gate_errors.get(inst[0].name, 0.0))
    return expected_success_rate

def compute_analytic_success_rate_estimate1(circuit, created_at=None):
    created_at_date = created_at.date() if created_at is not None else datetime.date.today()
    analytic_estimate1 = compute_accumulated_gate_error1(circuit, gate_errors = get_median_gate_errors(created_at_date))
    analytic_estimate2 = compute_accumulated_gate_error1(circuit, gate_errors = get_mean_gate_errors(created_at_date))
    return (analytic_estimate1 + analytic_estimate2) / 2

In [ ]:
trials = benchmark_db.query(
    select(RandomBooleanFunctionTrial)
        .where(
            RandomBooleanFunctionTrial._circuit_qpy != None,
            RandomBooleanFunctionTrial.counts != None
        )
        .options(joinedload(RandomBooleanFunctionTrial.problem))
        .order_by(func.random())
        .limit(1000)
)

In [ ]:
confusion_matrix = np.zeros((2, 2), dtype=int)
for trial in trials:
    analytic_estimate = compute_analytic_success_rate_estimate1(trial.circuit, trial.created_at)
    y = 1 if trial.calculate_success_rate() >= 0.5 else 0
    y_pred = 1 if analytic_estimate >= 0.5 else 0
    confusion_matrix[y, y_pred] += 1
    

print(f"Confusion Matrix:\n{confusion_matrix}")
tn, fp, fn, tp = confusion_matrix.ravel()
accuracy = (tp + tn) / np.sum(confusion_matrix)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

### Create Data Sets

In [ ]:
raise Exception("Comment this Exception out if you actually want to re-create the train/test split")
# designate training and test sets
num_train = 1500
num_test = 500
train_ids = []
test_ids = []
for i, trial in enumerate(benchmark_db.query(
    select(RandomBooleanFunctionTrial)
    .where(RandomBooleanFunctionTrial._circuit_qpy != None)
    .order_by(func.random())
    .limit(num_train + num_test)
    .options(joinedload(RandomBooleanFunctionTrial.problem))
)):
    if i < num_train:
        train_ids.append(trial.id)
    else:
        test_ids.append(trial.id)
        
# save datasets to disk for reproducibility
with open("train_ids.json", "w") as f:
    json.dump(train_ids, f)
with open("test_ids.json", "w") as f:
    json.dump(test_ids, f)

In [ ]:
# load data sets
def get_features(trial: RandomBooleanFunctionTrial) -> Tuple[np.array, int]:
    features = np.ones(7)
    features[1] = trial.circuit_num_qubits
    features[2] = trial.circuit_depth
    features[3] = trial.circuit_op_counts.get('ecr', 0)
    features[4] = trial.circuit_op_counts.get('rz', 0)
    features[5] = trial.circuit_op_counts.get('sx', 0)
    features[6] = trial.circuit_op_counts.get('x', 0)
    label = 1 if trial.calculate_success_rate() > 0.5 else 0
    return features, label

def get_features2(trial: RandomBooleanFunctionTrial) -> Tuple[np.array, int]:
    created_at = trial.created_at
    gate_errors = get_mean_gate_errors(created_at.date())

    features = np.ones(7)
    features[1] = trial.circuit_num_qubits
    features[2] = trial.circuit_depth
    features[3] = trial.circuit_op_counts.get('ecr', 0) * gate_errors.get('ecr', 0)
    features[4] = trial.circuit_op_counts.get('rz', 0) * gate_errors.get('rz', 0)
    features[5] = trial.circuit_op_counts.get('sx', 0) * gate_errors.get('sx', 0)
    features[6] = trial.circuit_op_counts.get('x', 0) * gate_errors.get('x', 0)
    label = 1 if trial.calculate_success_rate() > 0.5 else 0
    return features, label


def get_trials_by_ids(db: BenchmarkDatabase, ids: List[int]) -> List[RandomBooleanFunctionTrial]:
    return db.query(
        select(RandomBooleanFunctionTrial)
        .where(RandomBooleanFunctionTrial.id.in_(ids))
        .options(joinedload(RandomBooleanFunctionTrial.problem))
    )

train_ids = []
test_ids = []
with open("train_ids.json", "r") as f:
    train_ids = json.load(f)
with open("test_ids.json", "r") as f:
    test_ids = json.load(f)

# construct train data set
train_trials = get_trials_by_ids(benchmark_db, train_ids)
train_data = [get_features(trial) for trial in train_trials]
X = np.zeros((len(train_trials), len(train_data[0][0])))
Y = np.zeros((len(train_trials), 1))
for i, data in enumerate(train_data):
    X[i] = data[0]
    Y[i, 0] = data[1]

# construct test data set
test_trials = get_trials_by_ids(benchmark_db, test_ids)
test_data = [get_features(trial) for trial in test_trials]
X_test = np.zeros((len(test_trials), len(test_data[0][0])))
Y_test = np.zeros((len(test_trials), 1))
for i, data in enumerate(test_data):
    X_test[i] = data[0]
    Y_test[i, 0] = data[1]

# cache datasets to disk
np.save("train_features.npy", X)
np.save("train_labels.npy", Y)
np.save("test_features.npy", X_test)
np.save("test_labels.npy", Y_test)

### Learning Algorithm

In [ ]:
def logistic_regression_gd(X_aug, y, learning_rate=0.01, max_iter=1000):
    """
    Logistic regression using regular gradient descent
    """
    N, d = X_aug.shape
    w = np.zeros(d)

    errors = []

    for _ in range(max_iter):
        update_vec = np.zeros(d)
        for i in range(N):
            power = -y[i] * (w.T @ X_aug[i, :])
            update_vec += -y[i] * X_aug[i, :] * (np.exp(power) / (1 + np.exp(power)))
        w -= learning_rate * (update_vec / N)
        errors.append(np.mean(np.log(1+np.exp(-y * (X_aug @ w)))))

    plt.plot(range(len(errors)), errors)
    plt.show()
    plt.clf()

    return w

def logistic_regression_sgd(X_aug, y, learning_rate=0.01, max_iter=1000):
    """
    Logistic regression using stochastic gradient descent
    constraint: all weights must be negative besides the bias term (since each should impact the circuit success negatively)
    """
    # if y is 0/1, convert to -1/1
    y = np.where(y <= 0, -1, 1)

    N, d = X_aug.shape
    w = np.zeros(d)
    #errors = []
    for _ in range(max_iter):
        i = np.random.randint(0, N)
        power = -y[i] * (w.T @ X_aug[i, :])
        grad = -y[i] * X_aug[i, :] * (np.exp(power) / (1 + np.exp(power)))
        w -= learning_rate * grad
        w[1:] = np.minimum(w[1:], 0)  # weight constraints
        #errors.append(np.mean(np.log(1+np.exp(-y * (X_aug @ w)))))
    #plt.plot(range(len(errors)), errors)
    #plt.show()
    #plt.clf()
    return w


def logistic_regression_neg_gd(X, y, lr=0.01, epochs=1000):
    y_pm = 2*y - 1
    N, d = X.shape
    v = np.zeros(d)  # unconstrained
    b = 0.0
    for _ in range(epochs):
        grad_b_total = 0
        grad_v_total = np.zeros(d)
        for i in range(N):
            i = np.random.randint(0, N)
            w = -np.log1p(np.exp(v))  # -softplus(v)
            z = w @ X[i, :] + b
            s = 1 / (1 + np.exp(y_pm[i]*z))
            grad_z = -y_pm[i] * s
            grad_v = grad_z * X[i, :] * (-1/(1+np.exp(-v)))  # derivative of -softplus
            grad_b = grad_z
            grad_v_total += grad_v
            grad_b_total += grad_b

        v -= (grad_v_total / N) * lr
        b -= (grad_b_total / N) * lr
    w = -np.log1p(np.exp(v))
    w[0] += b[0]  # incorporate bias into weight vector
    return w

def logistic_regression_constrained_sgd(X_unc, X_neg, Y, lr=0.01, epochs=1000, lam=0.01):
    """
    Logistic regression with constraint that certain weights be negative.
    For features X, split into X_unc (unconstrained weights) and X_neg (negative weights) so that the prediction 
    is Y_hat = X_unc @ w_unc + X_neg @ w_neg + b
    where w_neg is all negative (via -softplus), and b is a bias term
    """
    Y = 2*Y - 1  # convert y 0/1 to -1/+1
    N, d_unc = X_unc.shape
    N, d_neg = X_neg.shape
    w_unc = np.zeros(d_unc)
    v_neg = np.zeros(d_neg)  # w_neg = -softplus(v_neg)
    b = 0.0

    for _ in range(epochs):
        i = np.random.randint(0, N)

        # softplus
        w_neg = -np.log1p(np.exp(v_neg))
        z = X_unc[i, :] @ w_unc + X_neg[i, :] @ w_neg + b

        # logistic loss grad
        grad_z = -Y[i] / (1 + np.exp(Y[i] * z))

        # grad wrt w_unc
        grad_unc = grad_z * X_unc[i]
        grad_unc += lam * w_unc  # L2 regularization

        # grad wrt v_neg
        sigmoid_v = 1/ (1 + np.exp(-v_neg))
        grad_neg = grad_z * X_neg[i] * (-sigmoid_v)
        grad_neg += lam * w_neg * (-sigmoid_v) # L2 regularization

        # grad wrt bias
        grad_b = grad_z

        # update
        w_unc -= lr * grad_unc
        v_neg -= lr * grad_neg
        b -= lr * grad_b

    w_neg = -np.log1p(np.exp(v_neg))
    return w_unc, w_neg, b

def logistic_regression_constrained_gd(X_unc, X_neg, Y, lr=0.01, epochs=1000, lam=0.01):
    """
    Logistic regression with constraint that certain weights be negative.
    For features X, split into X_unc (unconstrained weights) and X_neg (negative weights) so that the prediction 
    is Y_hat = X_unc @ w_unc + X_neg @ w_neg + b
    where w_neg is all negative (via -softplus), and b is a bias term
    """
    Y = 2*Y - 1  # convert y 0/1 to -1/+1
    N, d_unc = X_unc.shape
    N, d_neg = X_neg.shape
    w_unc = np.zeros(d_unc)
    v_neg = np.zeros(d_neg)  # w_neg = -softplus(v_neg)
    b = 0.0

    for _ in range(epochs):
        grad_unc_total = np.zeros(d_unc)
        grad_neg_total = np.zeros(d_neg)
        grad_b_total = 0.0

        for i in range(N):
            # softplus
            w_neg = -np.log1p(np.exp(v_neg))
            z = X_unc[i, :] @ w_unc + X_neg[i, :] @ w_neg + b

            # logistic loss grad
            grad_z = -Y[i] / (1 + np.exp(Y[i] * z))

            # grad wrt w_unc
            grad_unc = grad_z * X_unc[i]
            grad_unc += lam * w_unc  # L2 regularization
            grad_unc_total += grad_unc

            # grad wrt v_neg
            sigmoid_v = 1/ (1 + np.exp(-v_neg))
            grad_neg = grad_z * X_neg[i] * (-sigmoid_v)
            grad_neg += lam * w_neg * (-sigmoid_v) # L2 regularization
            grad_neg_total += grad_neg

            # grad wrt bias
            grad_b = grad_z
            grad_b_total += grad_b

        # update
        w_unc -= lr * (grad_unc_total / N)
        v_neg -= lr * (grad_neg_total / N)
        b -= lr * (grad_b_total / N)

    w_neg = -np.log1p(np.exp(v_neg))
    return w_unc, w_neg, b


def logistic_regression_with_freeze(X_unc, X_freeze, Y, w_freeze, lr=0.01, epochs=1000, lam=0.01):
    """
    Logistic regression with constraint that certain weights be fixed (not learned).
    For features X, split into X_unc (unconstrained weights) and X_freeze (frozen weights) so that the prediction 
    is Y_hat = X_unc @ w_unc + X_freeze @ w_freeze + b
    where b is a bias term
    """
    Y = 2*Y - 1  # convert y 0/1 to -1/+1
    N, d_unc = X_unc.shape
    N, d_freeze = X_freeze.shape
    w_unc = np.zeros(d_unc)
    b = 0.0

    for _ in range(epochs):
        i = np.random.randint(0, N)

        z = X_unc[i, :] @ w_unc + X_freeze[i, :] @ w_freeze + b

        # logistic loss grad
        grad_z = -Y[i] / (1 + np.exp(Y[i] * z))

        # grad wrt w_unc
        grad_unc = grad_z * X_unc[i]
        grad_unc += lam * w_unc  # L2 regularization

        # grad wrt bias
        grad_b = grad_z

        # update
        w_unc -= lr * grad_unc
        b -= lr * grad_b

    return w_unc, b
        

def logistic_regression_neg(X, y, lr=0.01, epochs=1000, lam=0.01):
    """
    Logistic regression with constraint: all weights <= 0
    using reparameterization w = -softplus(v)
    with L2 regularization strength 'lam'
    """
    y_pm = 2*y - 1
    N, d = X.shape
    v = np.zeros(d)
    b = 0.0

    for _ in range(epochs):
        i = np.random.randint(0, N)
        # reparameterize
        w = -np.log1p(np.exp(v))  # = -softplus(v)
        z = w @ X[i, :] + b

        # logistic loss grad
        s = 1 / (1 + np.exp(y_pm[i] * z))
        grad_z = -y_pm[i] * s

        # gradient wrt v (chain rule)
        sigmoid_v = 1 / (1 + np.exp(-v))
        grad_v = grad_z * X[i, :] * (-sigmoid_v)
        
        grad_v += -lam * w * sigmoid_v  # regularization

        grad_b = grad_z

        # update
        v -= lr * grad_v
        b -= lr * grad_b

    # final weights
    w = -np.log1p(np.exp(v))
    w[0] += b[0]  # incorporate bias into weight vector
    return w

def logistic_regression_neg_gd(X, y, lr=0.01, epochs=1000, lam=0.01):
    """
    Logistic regression with constraint: all weights <= 0
    using reparameterization w = -softplus(v)
    with L2 regularization strength 'lam'
    """
    y_pm = 2*y - 1
    N, d = X.shape
    v = np.zeros(d)
    b = 0.0

    for _ in range(epochs):
        grad_v_total = np.zeros(d)
        grad_b_total = 0.0

        for i in range(N):
            # reparameterize
            w = -np.log1p(np.exp(v))  # = -softplus(v)
            z = w @ X[i, :] + b

            # logistic loss grad
            s = 1 / (1 + np.exp(y_pm[i] * z))
            grad_z = -y_pm[i] * s

            sigmoid_v = 1 / (1 + np.exp(-v))
            grad_v = grad_z * X[i, :] * (-sigmoid_v)

            grad_v += -lam * w * sigmoid_v  # regularization

            grad_b = grad_z

            grad_v_total += grad_v
            grad_b_total += grad_b

        # update
        v -= lr * (grad_v_total / N)
        b -= lr * (grad_b_total / N)

    # final weights
    w = -np.log1p(np.exp(v))
    w[0] += b[0]  # incorporate bias into weight vector
    return w

In [ ]:
from sklearn.metrics import confusion_matrix, log_loss

def evaluate(X, Y, w):
    Y_pred = sigmoid(X @ w)
    confusion_matrix_in = confusion_matrix(np.round(Y), np.round(Y_pred))
    cross_entropy_in = log_loss(np.round(Y), Y_pred) / len(Y)
    print("In-sample Confusion Matrix:")
    print(confusion_matrix_in)
    print(f"(Cross Entropy): {cross_entropy_in}")
    percent_correct_in = np.trace(confusion_matrix_in) / np.sum(confusion_matrix_in)
    print(f"Percent Correct: {percent_correct_in * 100:.2f}%")

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

### Train

In [ ]:
from sklearn.preprocessing import StandardScaler
X = np.load("train_features.npy")
Y = np.load("train_labels.npy")
X_test = np.load("test_features.npy")
Y_test = np.load("test_labels.npy")

#scaler = StandardScaler()
#X = scaler.fit_transform(X)
#X_test = scaler.transform(X_test)

In [ ]:
# compute VIF measures
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
print(vif_data)

In [ ]:
# use partial negative constrained regression
w_unc, w_neg, b = logistic_regression_constrained_gd(X[:, 1:3], X[:, 3:], Y, lr=0.01, epochs=1000, lam=1)
w = np.concatenate((b, w_unc, w_neg))
print(w)
print("In-sample Evaluation:")
evaluate(X, Y, w)

print("Out-of-sample Evaluation:")
evaluate(X_test, Y_test, w)

In [ ]:
# use full negative constrained regression
w = logistic_regression_neg(X, Y, lr=0.01, epochs=10000, lam=0.1)
print(w)
print("In-sample Evaluation:")
evaluate(X, Y, w)

print("Out-of-sample Evaluation:")
evaluate(X_test, Y_test, w)

In [ ]:
# use frozen weights based on known backend parameters
avg_error = get_mean_gate_errors(datetime.date.today())
w_freeze = np.array([-avg_error.get(g, 0) for g in ["ecr", "rz", "sx", "x"]])
w_unc, b = logistic_regression_with_freeze(X[:, 1:3], X[:, 3:], Y, w_freeze, lr=0.002, epochs=400000, lam=2)
w = np.concatenate((b, w_unc, w_freeze))
print(w)
print("In-sample Evaluation:")
evaluate(X, Y, w)

print("Out-of-sample Evaluation:")
evaluate(X_test, Y_test, w)

In [ ]:
# use frozen weights based on known backend parameters
avg_error = get_median_gate_errors(datetime.date.today())
w_freeze = np.array([-avg_error.get(g, 0) for g in ["ecr", "rz", "sx", "x"]])
w_unc, b = logistic_regression_with_freeze(X[:, 1:3], X[:, 3:], Y, w_freeze, lr=0.002, epochs=400000, lam=2)
w = np.concatenate((b, w_unc, w_freeze))
print(w)
print("In-sample Evaluation:")
evaluate(X, Y, w)

print("Out-of-sample Evaluation:")
evaluate(X_test, Y_test, w)

### Evaluation

In [ ]:
print("In-sample Evaluation:")
evaluate(X, Y, w)

print("Out-of-sample Evaluation:")
evaluate(X_test, Y_test, w)

In [ ]:
# compare the label distributions
print(np.bincount(np.round(Y_test).ravel().astype(int)))

In [ ]:
print("Model coefficients interpretation:")
for (coef, name) in zip([b[0], *list(w)], 
                    ["Bias", "Num Qubits", "Circuit Depth", "ECR Count", "RZ Count", "SX Count", "X Count"]):
    print(f"{name}: {coef:.4f}")